In [0]:
import os
import cv2
import numpy as np
import tensorflow as tf

In [0]:
IMG_SIZE = 299 # All images will be resized to 160x160
############################################
def format_example(image):
  image = tf.cast(image, tf.float32)
  image = (image/127.5)-1
  image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
  return image
############################################
def convertVideo(videoDir):

  capture = cv2.VideoCapture(videoDir)
  frameCount = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
  frameWidth = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH))
  frameHeight = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
  buffer = np.empty((frameCount, frameHeight, frameWidth, 3), np.dtype('uint8'))
  
  fc = 0
  ret = True
  while (fc < frameCount  and ret):
    ret, buffer[fc] = capture.read()
    fc += 1 

  capture.release()
  
  tf_image_stack = tf.convert_to_tensor(buffer)
  tf_resize_image_stack = format_example(tf_image_stack)
  
  return tf_resize_image_stack 
############################################
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

# Create the base model from the pre-trained model Inception
base_model = tf.keras.applications.InceptionV3(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

maxLen = 100 ######

listed = np.arange(maxLen)
indices = []
for element in listed :
  indices.append([element])

#############################################
def extract_feature(video_tensor):
  
  feature_image = base_model(video_tensor)
  feature = global_average_layer(feature_image)

  frames , featureLen = feature.shape
  
  num_pad = max(0,maxLen - frames)
  padding = [[0,num_pad],[0,0]]
  padded_feature = tf.pad(feature,padding,'CONSTANT')
  cropped_padded_feature = tf.gather_nd(padded_feature,indices)

  return cropped_padded_feature 

################################################  
  

In [0]:
def feature_fromPath(videoDir) :
  tf_resize_image_stack = convertVideo(videoDir)
  feature_stack_padded = extract_feature(tf_resize_image_stack)

  return feature_stack_padded

In [0]:
directory = "drive/My Drive/VIDEO_RGB"
entries = os.listdir(directory)
#print(entries)

feature_block = []
label_block = []

for i in entries :  
  currDir = directory + '/' + i
  video_list = os.listdir(currDir)  
  print(i , "Folder Started")
  num = 1;

  for video in video_list:      
      videoDir = currDir + '/' + video
      f = feature_fromPath(videoDir)
      print("Video :",num,"Completed")
      feature_block.append(f)
      label_block.append(i)
      num = num + 1
  
  print(i , "Folder Complete") 
  



forehand_openstands Folder Started
Video : 1 Completed
Video : 2 Completed
Video : 3 Completed
Video : 4 Completed
Video : 5 Completed
Video : 6 Completed
Video : 7 Completed
Video : 8 Completed
Video : 9 Completed
Video : 10 Completed
Video : 11 Completed
Video : 12 Completed
Video : 13 Completed
Video : 14 Completed
Video : 15 Completed
Video : 16 Completed
Video : 17 Completed
Video : 18 Completed
Video : 19 Completed
Video : 20 Completed
Video : 21 Completed
Video : 22 Completed
Video : 23 Completed
Video : 24 Completed
Video : 25 Completed
Video : 26 Completed
Video : 27 Completed
Video : 28 Completed
Video : 29 Completed
Video : 30 Completed
Video : 31 Completed
Video : 32 Completed
Video : 33 Completed
Video : 34 Completed
Video : 35 Completed
Video : 36 Completed
Video : 37 Completed
Video : 38 Completed
Video : 39 Completed
Video : 40 Completed
Video : 41 Completed
Video : 42 Completed
Video : 43 Completed
Video : 44 Completed
Video : 45 Completed
Video : 46 Completed
Video : 

In [0]:
DataSet_Features = tf.stack(feature_block)
DataSet_Labels = tf.stack(label_block)


In [0]:
x = DataSet_Features.numpy()
y = DataSet_Labels.numpy()

np.save("drive/My Drive/VIDEO_RGB/DataFeatures.npy",x,allow_pickle=True)
np.save("drive/My Drive/VIDEO_RGB/Labels.npy",y,allow_pickle=True)


In [0]:
X = np.load("drive/My Drive/VIDEO_RGB/DataFeatures.npy",allow_pickle=True)
Y = np.load("drive/My Drive/VIDEO_RGB/Labels.npy",allow_pickle=True)

X = tf.convert_to_tensor(X)
Y = tf.convert_to_tensor(Y)
